# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [34]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, stddev, col, concat, desc, explode, lit, min, max, split, udf, sum, regexp_replace
from pyspark.sql.types import IntegerType, StringType

from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier, GBTClassifier, LogisticRegression, MultilayerPerceptronClassifier,LinearSVC, RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.feature import CountVectorizer, IDF, Normalizer, PCA, RegexTokenizer, StandardScaler, StopWordsRemover, StringIndexer, VectorAssembler, IndexToString
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import time
import re
import datetime
import pickle
from pyspark.sql import Window

In [2]:
# create a Spark session
spark = SparkSession.builder \
    .master("local") \
    .appName("Sparkify Project") \
    .getOrCreate()

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [3]:
stack_overflow_data = 'mini_sparkify_event_data.json'
df = spark.read.json(stack_overflow_data)
df.persist()

DataFrame[artist: string, auth: string, firstName: string, gender: string, itemInSession: bigint, lastName: string, length: double, level: string, location: string, method: string, page: string, registration: bigint, sessionId: bigint, song: string, status: bigint, ts: bigint, userAgent: string, userId: string]

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

In [4]:
df.show()

+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+
|              artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|           page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+
|      Martha Tilston|Logged In|    Colin|     M|           50| Freeman|277.89016| paid|     Bakersfield, CA|   PUT|       NextSong|1538173362000|       29|           Rockpools|   200|1538352117000|Mozilla/5.0 (Wind...|    30|
|    Five Iron Frenzy|Logged In|    Micah|     M|           79|    Long|236.09424| free|Bost

### Observe account types

In [20]:
df.dropDuplicates()\
    .groupBy('level')\
    .count()\
    .show()

+-----+------+
|level| count|
+-----+------+
| free| 58338|
| paid|228162|
+-----+------+



there are two types of the accounts are existed: free and paid

### Observe auth column

In [21]:
df.dropDuplicates()\
    .groupBy('auth')\
    .count()\
    .show()

+----------+------+
|      auth| count|
+----------+------+
|Logged Out|  8249|
| Cancelled|    52|
|     Guest|    97|
| Logged In|278102|
+----------+------+



### Observe Method column

In [22]:
df.dropDuplicates()\
    .groupBy('method')\
    .count()\
    .show()

+------+------+
|method| count|
+------+------+
|   PUT|261064|
|   GET| 25436|
+------+------+



two types of method: put and get which could be used as a features for classification

### Observe Page column

In [23]:
df.dropDuplicates()\
    .groupBy('page')\
    .count()\
    .show()

+--------------------+------+
|                page| count|
+--------------------+------+
|              Cancel|    52|
|    Submit Downgrade|    63|
|         Thumbs Down|  2546|
|                Home| 14457|
|           Downgrade|  2055|
|         Roll Advert|  3933|
|              Logout|  3226|
|       Save Settings|   310|
|Cancellation Conf...|    52|
|               About|   924|
| Submit Registration|     5|
|            Settings|  1514|
|               Login|  3241|
|            Register|    18|
|     Add to Playlist|  6526|
|          Add Friend|  4277|
|            NextSong|228108|
|           Thumbs Up| 12551|
|                Help|  1726|
|             Upgrade|   499|
+--------------------+------+
only showing top 20 rows



page column reflect multiple actions of the users

### User Agent

In [17]:
df.dropDuplicates()\
    .groupBy('userAgent')\
    .count()\
    .show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------+-----+
|userAgent                                                                                                                                  |count|
+-------------------------------------------------------------------------------------------------------------------------------------------+-----+
|"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"                 |240  |
|"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"                                   |5238 |
|Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:31.0) Gecko/20100101 Firefox/31.0                                                                 |62   |
|"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Saf

In [51]:
df.select('userAgent').distinct().count()

57

userAgent include the os system and softwares; it has 57 types combinations so this would be considered as a categorical features

### UserId

In [31]:
df.select('userId','auth').sort('userId').show(20000,truncate=False)

+------+----------+
|userId|auth      |
+------+----------+
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|
|      |Logged Out|


The data include a lot of empty userId which is for the user logged out/guests. These unknown records are difficult to track and they cannot be used to predict the downgrade. Therefore, the empty records will be removed.

In [4]:
df = df.where(df['userId']!='')

In [5]:
df.dropDuplicates()\
    .groupBy('auth')\
    .count()\
    .show()

+---------+------+
|     auth| count|
+---------+------+
|Cancelled|    52|
|Logged In|278102|
+---------+------+



In [34]:
df.where(df.auth=='Cancelled').select('page').show(truncate=False)

+-------------------------+
|page                     |
+-------------------------+
|Cancellation Confirmation|
|Cancellation Confirmation|
|Cancellation Confirmation|
|Cancellation Confirmation|
|Cancellation Confirmation|
|Cancellation Confirmation|
|Cancellation Confirmation|
|Cancellation Confirmation|
|Cancellation Confirmation|
|Cancellation Confirmation|
|Cancellation Confirmation|
|Cancellation Confirmation|
|Cancellation Confirmation|
|Cancellation Confirmation|
|Cancellation Confirmation|
|Cancellation Confirmation|
|Cancellation Confirmation|
|Cancellation Confirmation|
|Cancellation Confirmation|
|Cancellation Confirmation|
+-------------------------+
only showing top 20 rows



'Cancelled' in 'auth' column is equivalent to the 'Cancellation Confirmation' in 'page' column

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.



In [6]:
#define the flag event udf to transform event to 0 or 1
flag_event = udf(lambda x : 1 if x in {'Cancellation Confirmation', 'Downgrade'} else 0, IntegerType())
#define the current churn or not state
df = df.withColumn('Churn',flag_event('page'))
df.where(df.Churn == 1).show()

+------+---------+---------+------+-------------+----------+------+-----+--------------------+------+---------+-------------+---------+----+------+-------------+--------------------+------+-----+
|artist|     auth|firstName|gender|itemInSession|  lastName|length|level|            location|method|     page| registration|sessionId|song|status|           ts|           userAgent|userId|Churn|
+------+---------+---------+------+-------------+----------+------+-----+--------------------+------+---------+-------------+---------+----+------+-------------+--------------------+------+-----+
|  null|Logged In|    Alexi|     F|            4|    Warren|  null| paid|Spokane-Spokane V...|   GET|Downgrade|1532482662000|       53|null|   200|1538354749000|Mozilla/5.0 (Wind...|    54|    1|
|  null|Logged In|    Faigy|     F|           29|      Howe|  null| paid|Phoenix-Mesa-Scot...|   GET|Downgrade|1538211832000|      248|null|   200|1538373286000|Mozilla/5.0 (Maci...|    95|    1|
|  null|Logged In|  

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [7]:
# clearly represent registration and timestamp by creating new columns
new_ts = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).strftime("%m-%d-%Y %H:%M:%S"))
df = df.withColumn('updated_ts', new_ts('ts'))


In [8]:
# label users who have churned
churn_window = Window.partitionBy("userId").rangeBetween(Window.unboundedPreceding, Window.unboundedFollowing)
df = df.withColumn("user_churned", sum('Churn').over(churn_window))
df.show()

+--------------------+---------+---------+------+-------------+---------+---------+-----+--------------------+------+-----------+-------------+---------+--------------------+------+-------------+--------------------+------+-----+-------------------+------------+
|              artist|     auth|firstName|gender|itemInSession| lastName|   length|level|            location|method|       page| registration|sessionId|                song|status|           ts|           userAgent|userId|Churn|         updated_ts|user_churned|
+--------------------+---------+---------+------+-------------+---------+---------+-----+--------------------+------+-----------+-------------+---------+--------------------+------+-------------+--------------------+------+-----+-------------------+------------+
|Sleeping With Sirens|Logged In| Darianna|     F|            0|Carpenter|202.97098| free|Bridgeport-Stamfo...|   PUT|   NextSong|1538016340000|       31|Captain Tyin Knot...|   200|1539003534000|"Mozilla/5.0 (iP

In [27]:
df.where(df.user_churned==1).select('page','user_churned','Churn','updated_ts').show()

+--------------------+------------+-----+-------------------+
|                page|user_churned|Churn|         updated_ts|
+--------------------+------------+-----+-------------------+
|            NextSong|           1|    0|10-12-2018 04:05:44|
|            NextSong|           1|    0|10-12-2018 04:11:21|
|            NextSong|           1|    0|10-12-2018 04:15:11|
|            NextSong|           1|    0|10-12-2018 04:18:34|
|            NextSong|           1|    0|10-12-2018 04:22:04|
|            NextSong|           1|    0|10-12-2018 04:28:35|
|            NextSong|           1|    0|10-12-2018 04:32:08|
|            NextSong|           1|    0|10-12-2018 04:35:06|
|         Roll Advert|           1|    0|10-12-2018 04:35:17|
|              Cancel|           1|    0|10-12-2018 04:35:18|
|Cancellation Conf...|           1|    1|10-12-2018 04:35:18|
|            NextSong|           1|    0|10-11-2018 00:04:35|
|         Roll Advert|           1|    0|10-11-2018 00:06:18|
|       

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

### 1. User Agent

In [9]:
indexer = StringIndexer(inputCol="userAgent", outputCol="userAgent_num")
indexed_df = indexer.fit(df).transform(df)

In [10]:
indexed_df = indexed_df.withColumn("userAgent_num", indexed_df["userAgent_num"].cast(IntegerType()))

In [11]:
userAgent_df = indexed_df.groupBy("userId").pivot("userAgent_num").agg(lit(1))
userAgent_df = userAgent_df.na.fill(0)

In [38]:
userAgent_df.show()

+------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|userId|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9| 10| 11| 12| 13| 14| 15| 16| 17| 18| 19| 20| 21| 22| 23| 24| 25| 26| 27| 28| 29| 30| 31| 32| 33| 34| 35| 36| 37| 38| 39| 40| 41| 42| 43| 44| 45| 46| 47| 48| 49| 50| 51| 52| 53| 54| 55|
+------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|100010|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|
|200002|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|

### 2. Gender

In [12]:
gender_df = df.groupBy("userId").pivot("gender").agg(lit(1))
gender_df = gender_df.na.fill(0)
gender_df.show()

+------+---+---+
|userId|  F|  M|
+------+---+---+
|100010|  1|  0|
|200002|  0|  1|
|   125|  0|  1|
|    51|  0|  1|
|     7|  0|  1|
|   124|  1|  0|
|    15|  0|  1|
|    54|  1|  0|
|   155|  1|  0|
|100014|  0|  1|
|   154|  1|  0|
|   132|  1|  0|
|   101|  0|  1|
|    11|  1|  0|
|   138|  0|  1|
|300017|  1|  0|
|100021|  0|  1|
|    69|  1|  0|
|    29|  0|  1|
|    42|  1|  0|
+------+---+---+
only showing top 20 rows



### 3. Level

In [13]:
level_df = df.groupBy("userId").pivot("level").agg(lit(1)).na.fill(0)
level_df.show()

+------+----+----+
|userId|free|paid|
+------+----+----+
|200002|   1|   1|
|100010|   1|   0|
|   125|   1|   0|
|     7|   1|   0|
|   124|   0|   1|
|    51|   0|   1|
|    54|   1|   1|
|    15|   0|   1|
|   155|   1|   1|
|   132|   1|   1|
|   154|   1|   0|
|100014|   0|   1|
|    11|   1|   1|
|   101|   1|   1|
|300017|   0|   1|
|   138|   1|   1|
|100021|   1|   0|
|    29|   1|   1|
|    69|   1|   1|
|   112|   1|   0|
+------+----+----+
only showing top 20 rows



### 4. Number of songs listened

In [14]:
num_songs_df = df.select('userId', 'song').groupBy('userId').count().withColumnRenamed('count', 'num_songs')
num_songs_df.show()

+------+---------+
|userId|num_songs|
+------+---------+
|100010|      381|
|200002|      474|
|   125|       11|
|    51|     2464|
|   124|     4825|
|     7|      201|
|    54|     3437|
|    15|     2278|
|   155|     1002|
|   132|     2304|
|   154|      118|
|100014|      310|
|   101|     2149|
|    11|      848|
|   138|     2469|
|300017|     4428|
|    29|     3603|
|    69|     1342|
|100021|      319|
|    42|     4257|
+------+---------+
only showing top 20 rows



### 5. average lenth of song for each user

In [15]:
song_length_df = df.where(df.page=='NextSong').select('userId', 'sessionId', 'length')
song_length_df = song_length_df.withColumn('hours', (song_length_df.length / 3600))
song_length_df = song_length_df.groupBy('userId','sessionId').sum('hours')
song_length_df = song_length_df.groupBy('userId').agg(avg('sum(hours)').alias('mean_hours'), stddev('sum(hours)').alias('stdev_hours')).na.fill(0)

In [43]:
song_length_df.show()

+------+------------------+------------------+
|userId|        mean_hours|       stdev_hours|
+------+------------------+------------------+
|100010|2.6563848154761907|1.5940891633984364|
|200002|       4.352262775| 3.481087253628621|
|   125|0.5803091944444445|               0.0|
|    51|14.535440077777778|11.600663366034107|
|   124|10.042778698313494| 9.370878101268762|
|     7|1.5092891706349203|1.7372572756564078|
|    54| 5.645594599523808|6.7241639034102025|
|    15| 9.470389004166666| 6.159367750898716|
|   155|  9.20274499537037| 6.528671599395436|
|100014| 3.134420003703703|1.7543652306205215|
|   132| 8.387480969444443| 8.411844077944691|
|   154|1.9129651768518519|0.8567096079055881|
|   101|12.429555963055552| 14.92789006740804|
|    11| 2.772047969444445| 2.754383050218178|
|300017| 4.086552733196721|3.6120139088908414|
|   138| 9.489812643703699|11.718117293713126|
|    29| 6.164359171323531|   8.7578661203911|
|    69|  8.82913659382716| 8.270113865858235|
|100021| 3.20

### 6. counts of ThumbsUp , ThumbsDown etc

In [16]:
# create the event counts table
user_page_df = df.groupby('userId').pivot('page').count().na.fill(0)
# drop the labels in case of overfitting
user_page_df = user_page_df.drop(*['Cancellation Confirmation', 'Downgrade'])

In [153]:
user_page_df.show()

+------+-----+----------+---------------+------+-----+----+----+------+--------+-----------+-------------+--------+----------------+--------------+-----------+---------+-------+
|userId|About|Add Friend|Add to Playlist|Cancel|Error|Help|Home|Logout|NextSong|Roll Advert|Save Settings|Settings|Submit Downgrade|Submit Upgrade|Thumbs Down|Thumbs Up|Upgrade|
+------+-----+----------+---------------+------+-----+----+----+------+--------+-----------+-------------+--------+----------------+--------------+-----------+---------+-------+
|200002|    3|         4|              8|     0|    0|   2|  20|     5|     387|          7|            0|       3|               0|             1|          6|       21|      2|
|100010|    1|         4|              7|     0|    0|   2|  11|     5|     275|         52|            0|       0|               0|             0|          5|       17|      2|
|   125|    0|         0|              0|     1|    0|   0|   0|     0|       8|          1|            0|    

### 7. Number of artist the user follow

In [17]:
# Number of artists the user listen to
num_artists_user_df = df.select('userId', 'artist').dropDuplicates().groupBy('userId').count().withColumnRenamed("count", "num_aritsts")
num_artists_user_df.show()

+------+-----------+
|userId|num_aritsts|
+------+-----------+
|100010|        253|
|200002|        340|
|   125|          9|
|    51|       1386|
|   124|       2233|
|     7|        143|
|    15|       1303|
|    54|       1745|
|   155|        644|
|   132|       1300|
|   154|         79|
|100014|        234|
|   101|       1242|
|    11|        535|
|   138|       1333|
|300017|       2071|
|    29|       1805|
|    69|        866|
|100021|        208|
|    42|       2074|
+------+-----------+
only showing top 20 rows



### 8. Sessions count per user

In [18]:
num_sessions_user_df = df.select('userId', 'sessionId').dropDuplicates().groupby('userId').count().withColumnRenamed('count', 'num_sessions')
num_sessions_user_df.show()

+------+------------+
|userId|num_sessions|
+------+------------+
|100010|           7|
|200002|           6|
|   125|           1|
|    51|          10|
|   124|          29|
|     7|           7|
|    54|          37|
|    15|          15|
|   155|           6|
|100014|           6|
|   132|          16|
|   154|           3|
|   101|          10|
|    11|          16|
|300017|          63|
|   138|          15|
|    29|          34|
|    69|           9|
|100021|           5|
|    42|          47|
+------+------------+
only showing top 20 rows



### 9. The length of user's subscription

In [19]:

# time of users registration
reg_ts = df.select('userId', 'registration').dropDuplicates().withColumnRenamed('registration', 'start')
# calculate the maximal timestamp as the end
end_ts = df.groupBy('userId').max('ts').withColumnRenamed('max(ts)', 'end')
# The difference
reg_df = reg_ts.join(end_ts,'userId')
ticks_per_day = 1000 * 60 * 60 * 24 # as the timestamp is in ticks (0.001 seconds)
# Merge in one df
reg_df = reg_df.select('userId', ((reg_df.end-reg_df.start)/ticks_per_day).alias('subscription_length'))


reg_df.show()

+------+-------------------+
|userId|subscription_length|
+------+-------------------+
|200024|  28.42769675925926|
|200025|  116.9954861111111|
|   118|  63.12707175925926|
|    28|  44.31443287037037|
|100003| 22.748113425925926|
|   122|  52.81670138888889|
|   149| 46.179710648148145|
|   111|  82.22630787037038|
|300001|  188.3117824074074|
|     3|  80.89527777777778|
|    62| 134.06662037037037|
|100017|  78.88440972222222|
|    10|  51.76265046296297|
|200010| 38.142488425925926|
|    40|  78.96927083333334|
|   131| 120.89991898148148|
|   136|  75.35636574074074|
|    75|  68.92494212962963|
|   128|  95.33168981481481|
|200019|  56.23005787037037|
+------+-------------------+
only showing top 20 rows



### 10. Location

In [20]:
location_df = df.select(col('userId'),split(col("location"),",")[1]).groupBy("userId").pivot("split(location, ,)[1]").agg(lit(1))
location_df = location_df.na.fill(0)
location_df.show()

+------+---+---+---+---+---+---+---+------------+---+---+------+---+---+---------+------+---+---+------+---+------+---+------+---+------+------+------+---+---+---+------+------+---+---+---+---+---------+---+---------+---+---+------+---+------+------------+------+---+------+---+---------+------+---+---+------+---+------+---+---+---+
|userId| AK| AL| AR| AZ| CA| CO| CT| DC-VA-MD-WV| FL| GA| GA-AL| IA| IL| IL-IN-WI| IL-MO| IN| KY| KY-IN| LA| MA-NH| MD| MD-WV| MI| MN-WI| MO-IL| MO-KS| MS| MT| NC| NC-SC| NE-IA| NH| NJ| NV| NY| NY-NJ-PA| OH| OH-KY-IN| OK| OR| OR-WA| PA| PA-NJ| PA-NJ-DE-MD| RI-MA| SC| SC-NC| TN| TN-MS-AR| TN-VA| TX| UT| UT-ID| VA| VA-NC| WA| WI| WV|
+------+---+---+---+---+---+---+---+------------+---+---+------+---+---+---------+------+---+---+------+---+------+---+------+---+------+------+------+---+---+---+------+------+---+---+---+---+---------+---+---------+---+---+------+---+------+------------+------+---+------+---+---------+------+---+---+------+---+------+---+---+---

### 11. labels

In [21]:
# setting up the label
label = df.select('userId', col('Churn').alias('label')).dropDuplicates()

In [52]:
label.groupby('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1|  171|
|    0|  225|
+-----+-----+



### Gather data

In [22]:
# numeric features
numeric_features = num_songs_df.join(user_page_df,on='userId',how='outer')\
                            .join(song_length_df,'userId','outer') \
                            .join(num_artists_user_df,'userId','outer') \
                            .join(num_sessions_user_df,'userId','outer') \
                            .join(reg_df,'userId','outer') 
assembler = VectorAssembler(inputCols=numeric_features.columns[1:], outputCol="Numerical_Features")
numeric_features = assembler.transform(numeric_features)
scaler2 = StandardScaler(inputCol="Numerical_Features", outputCol="Numeric_Features", withStd=True)
scalerModel = scaler2.fit(numeric_features)
numeric_features = scalerModel.transform(numeric_features)
numeric_features.select('Numeric_Features').show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Numeric_Features                                                                                                                                                                                                                                                                                                                                                                                                                           |
+-------------------------------------------------------------------------------------------------------------------------------------------

In [23]:
# categorial features
categorical_features  = userAgent_df.join(gender_df,'userId','outer') \
                                    .join(level_df,'userId','outer') \
                                    .join(location_df,'userId','outer')
categorical_features.show(truncate=False)

+------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+----+---+---+---+---+---+---+---+------------+---+---+------+---+---+---------+------+---+---+------+---+------+---+------+---+------+------+------+---+---+---+------+------+---+---+---+---+---------+---+---------+---+---+------+---+------+------------+------+---+------+---+---------+------+---+---+------+---+------+---+---+---+
|userId|0  |1  |2  |3  |4  |5  |6  |7  |8  |9  |10 |11 |12 |13 |14 |15 |16 |17 |18 |19 |20 |21 |22 |23 |24 |25 |26 |27 |28 |29 |30 |31 |32 |33 |34 |35 |36 |37 |38 |39 |40 |41 |42 |43 |44 |45 |46 |47 |48 |49 |50 |51 |52 |53 |54 |55 |F  |M  |free|paid| AK| AL| AR| AZ| CA| CO| CT| DC-VA-MD-WV| FL| GA| GA-AL| IA| IL| IL-IN-WI| IL-MO| IN| KY| KY-IN| LA| MA-NH| MD| MD-WV| MI| MN-WI| MO-IL| MO-KS| MS| MT| NC| NC-SC| NE-IA| NH| 

In [24]:
data = categorical_features.join(numeric_features.select('userId','Numeric_Features'),on='userId',how='outer')\
                            .join(label,on='userId',how='outer').drop('userId') \
                            .fillna(0)
data.show(1,truncate=False)

+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+----+---+---+---+---+---+---+---+------------+---+---+------+---+---+---------+------+---+---+------+---+------+---+------+---+------+------+------+---+---+---+------+------+---+---+---+---+---------+---+---------+---+---+------+---+------+------------+------+---+------+---+---------+------+---+---+------+---+------+---+---+---+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|0  |1  |2  |3  |4  |5  |6  |7  |8  |9  |10 |11 |12 |13 |14 |15 |16 |17 

In [25]:
vector = VectorAssembler(inputCols=categorical_features.columns[1:]+['Numeric_Features'], outputCol='features')
data = vector.transform(data)
data.show(1,truncate=False)

+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+----+---+---+---+---+---+---+---+------------+---+---+------+---+---+---------+------+---+---+------+---+------+---+------+---+------+------+------+---+---+---+------+------+---+---+---+---+---------+---+---------+---+---+------+---+------+------------+------+---+------+---+---------+------+---+---+------+---+------+---+---+---+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+-------------------------------------------------------------------------

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [26]:
data = data.select('label','features')
data.show(1,truncate=False)

+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                                                                                                                                                                                                                                                                                                        |
+-----+---------------------------------------------------------------------------------------------------------------

##### Principle Component Analysis
This project uses PCA to eliminate redundant features and keep the informatic features

In [35]:
pca = PCA(k=20, inputCol="features")
pca.setOutputCol("pca_features")
model = pca.fit(data)
data = model.transform(data)
data = data.select('label',col('pca_features').alias('features'))

In [36]:
train,test = data.randomSplit([0.8, 0.2], seed=50)
train.persist()
test.persist()

DataFrame[label: int, features: vector]

In [28]:
def build_up_model(train,model_filepath,estimator,paramGrid):
    '''
    This function is to build up model for churn classification
    INPUT:
        train - training set
        file_name - file name to save as
        pipeline - model pipeline
        paramGrid - the grid search parameters
    OUTPUT:
        model - trained classifier
    '''
    start = time.time()
    # cross validation
    crossval = CrossValidator(estimator=estimator,
                              estimatorParamMaps=paramGrid,
                              evaluator=BinaryClassificationEvaluator(),
                              numFolds=3)
    

    # Fitting models
    print('Training model...')
    model = crossval.fit(train)
    
    # save the model
    try:
        model.save(model_filepath)
    except:
        # Overwrite if exists
        model.write().overwrite().save(model_filepath)
        print ('*Overwritten* ', end='')

    end = time.time()
    print('Trained model saved!')
    print(f'the model takes {round(end-start,4)} time')
    return model

In [29]:
def evaluation(model,test):
    results = model.transform(test)
    model_results = {}
    evaluator=MulticlassClassificationEvaluator()
    model_results['f1_score'] = evaluator.evaluate(results.select('label','prediction'),{evaluator.metricName: 'f1'})
    model_results['accuracy'] = evaluator.evaluate(results.select('label','prediction'), {evaluator.metricName: "accuracy"})
    print(model_results)
    

### Logistic Regression Model

In [138]:
model =  LogisticRegression()
# setup parameters of grid search
param_grid = ParamGridBuilder() \
    .addGrid(model.regParam,[0.0, 0.5]) \
    .build()
clf_LR = build_up_model(train,'LogisticRegression.pkl',model,param_grid)
evaluation(clf_LR,test)

Training model...
*Overwritten* Trained model saved!
the model takes 2181.7399 time
{'f1_score': 0.47153574367165196, 'accuracy': 0.5352112676056338}


### Decision Tree

In [141]:
model =  DecisionTreeClassifier()
# setup parameters of grid search
param_grid = ParamGridBuilder() \
    .addGrid(model.maxDepth,[3, 10]) \
    .addGrid(model.impurity,['entropy', 'gini']) \
    .build()
clf_DT = build_up_model(train,'DecisionTree.pkl',model,param_grid)
evaluation(clf_DT,test)

Training model...
Trained model saved!
the model takes 1156.8311 time
{'f1_score': 0.26330615270570795, 'accuracy': 0.3380281690140845}


### Gradient Boost Tree

In [32]:
model = GBTClassifier()
param_grid = ParamGridBuilder() \
    .addGrid(model.maxDepth,[3, 10]) \
    .addGrid(model.maxBins ,[3, 10]) \
    .build()
clf_GBT = build_up_model(train,'GradientBoostTree.pkl',model,param_grid)
evaluation(clf_GBT,test)

Training model...
Trained model saved!
the model takes 9996.7684 time
{'f1_score': 0.18580945295329965, 'accuracy': 0.19718309859154928}


### Random Forest model

In [37]:
model = RandomForestClassifier()
param_grid = ParamGridBuilder() \
    .addGrid(model.maxDepth,[3, 10]) \
    .addGrid(model.impurity,['entropy', 'gini']) \
    .addGrid(model.numTrees ,[20, 10]) \
    .addGrid(model.featureSubsetStrategy ,['sqrt', 'onethird']) \
    .build()
clf_RF = build_up_model(train,'RandomForest.pkl',model,param_grid)
evaluation(clf_RF,test)

Training model...
Trained model saved!
the model takes 3525.921 time
{'f1_score': 0.34999742041995563, 'accuracy': 0.39436619718309857}


### Multilayer Perceptron Classifier

In [44]:
model = MultilayerPerceptronClassifier()
param_grid = ParamGridBuilder() \
    .addGrid(model.stepSize ,[0.01, 0.02]) \
    .addGrid(model.layers, [[20, 10]]) \
    .build()
clf_MPC = build_up_model(train,'RandomForest.pkl',model,param_grid)
evaluation(clf_MPC,test)

Training model...
*Overwritten* Trained model saved!
the model takes 3122.0019 time
{'f1_score': 0.42080832823025116, 'accuracy': 0.4647887323943662}


### Linear Vector Machine

In [45]:
model = LinearSVC()
param_grid = ParamGridBuilder() \
    .addGrid(model.regParam ,[0.1, 0.5]) \
    .build()
clf_MPC = build_up_model(train,'LinearVectorMachine.pkl',model,param_grid)
evaluation(clf_MPC,test)



Training model...
Trained model saved!
the model takes 6077.1745 time
{'f1_score': 0.35706833594157533, 'accuracy': 0.5211267605633803}


# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.